In [31]:
import pandas as pd
import numpy as np 
import requests 
from bs4 import BeautifulSoup
import time


In [58]:
#url = "https://coolblue.be/nl/"
#test_url = "https://www.coolblue.be/nl/laptops?pagina=1"
columns = ['Product_Name', 'Price', 'Review', 'Number_Of_Reviews']
columns2 = ['Product_Name', 'Price', 'Review', 'Number_Of_Reviews', 'Category']

In [4]:
category_laptops = ["laptops", "desktops", "monitoren", "schoudertassen","laptopstandaarden", "public-displays", "laptops/tweedekans", "tablets" ]
last_extension = "?pagina=1"

#Nota te nemen dat het nummer wel nog veranderd moet worden! 

In [2]:
# Assortiment v/d hoofdcategoriën!! 
url2 = "https://www.coolblue.be/nl/ons-assortiment"
html_cat = requests.get(url2).content
soup_cat = BeautifulSoup(html_cat, 'html.parser')
cats = soup_cat.find_all('h2')
cats2 = [categorie.getText().strip("\n").strip() for categorie in cats]
subcats = soup_cat.find_all(class_='category-navigation__item')
subcats = [subcategorie.getText().strip("\n").strip() for subcategorie in subcats]


In [3]:
def url (category, number):
    urls = []
    for i in range(1, number+1): 
        new_url = "https://www.coolblue.be/nl/"+category+"?pagina="+str(i)
        urls.append(new_url)
    return urls


In [4]:
def list_articles (soups): 
    name_article = [element.text for element in soups.findAll("div", {"class": "grid-section-xs--gap-2 js-product-item-title-wrapper"})]
    article_rows = [row.strip("\n").replace("\n", "") for row in name_article]
    return article_rows 


In [6]:
def list_prices (soups): 
    price = [element.text for element in soups.findAll("div", {"class": "grid-section-xs--gap-3 flex-container--justify-content--space-between"})]
    price_row = [row.strip("\n").replace("\n", "") for row in price]
    price_test = [row.split("-") for row in price_row]
    price_test2 = [row[0]+"00" for row in price_test]
    return price_test2

In [36]:
def list_stars (soups): 
    rating = [element.text for element in soups.findAll("div", {"class": "review-rating"})]
    rating_row = [row.strip("\n").replace("\n", "") for row in rating]
    rating_row2 = [row.strip(" ").strip("reviews") for row in rating_row]
    rating_row3 = [row.split("/") for row in rating_row2]
    rating_row4 = [row[0] for row in rating_row3]
    return rating_row4


In [37]:
def list_number_ratings (soups): 
    rating = [element.text for element in soups.findAll("div", {"class": "review-rating"})]
    rating_row = [row.strip("\n").replace("\n", "") for row in rating]
    rating_row2 = [row.strip(" ").strip("reviews") for row in rating_row]
    rating_row3 = [row.split("/") for row in rating_row2]
    review = [review[-1][-3:] for review in rating_row3]
    return review

In [38]:
def coolblue_v2 (): 
    cat = input("Which category you want to scrape? ")
    number_pages = input("How many pages you want to scrape? ")
    int_number_pages = int(number_pages)
    webpages = url(cat, int_number_pages)
    products2 = []
    for webpage in webpages: 
        html = requests.get(webpage).content
        soup = BeautifulSoup(html, 'html.parser')
        articles = list_articles(soup)
        prices = list_prices(soup)
        stars = list_stars(soup)
        num_ratings = list_number_ratings (soup)
        for i in range(0, len(articles)): 
            pro = [articles[i], prices[i], stars[i], num_ratings[i]]
            products2.append(pro)
    return products2


#monitoren
#schoudertassen

In [24]:
data = coolblue_v2()

df = pd.DataFrame(data, columns=colnames) 


In [56]:
def coolblue_v3 (): 
    producten = []
    for sub in subcats:
        try:  
            urllen = url(sub, 99)
            for weburl in urllen: 
                html = requests.get(weburl).content
                soup = BeautifulSoup(html, 'html.parser')
                articles = list_articles(soup)
                prices = list_prices(soup)
                stars = list_stars(soup)
                num_ratings = list_number_ratings (soup)
                for i in range(0, len(articles)): 
                    try: 
                        pro = [articles[i], prices[i], stars[i], num_ratings[i], sub]
                        producten.append(pro)
                    except: 
                        continue
            time.sleep(2)
        except: 
            continue
    return producten

    




In [71]:
bert_test_v5 = coolblue_v3()

In [72]:
data_v5 = bert_test_v5
colnames = columns
df_v5 = pd.DataFrame(data_v5, columns=colnames2)

In [76]:
df_v5.Category.unique()

array(['Laptops', 'Desktops', 'Monitoren', 'Tablets', 'E-readers',
       'Videokaarten', 'Printers', 'Toetsenborden', 'Muizen',
       'Geheugenkaarten', 'NAS', 'Routers', 'Switches', 'Consoles',
       'Games', 'Televisies', 'Beamers', 'Projectieschermen',
       'Koptelefoons', 'Oordopjes', 'Afstandsbedieningen',
       'Voicerecorders', 'Soundbars', 'Mediaspelers', 'Mengpanelen',
       'Keyboards', 'Synthesizers', 'Smartphones', 'Autoladers',
       'Sporthorloges', 'Smartwatches', 'Powerbanks', 'Wasmachines',
       'Droogkasten', 'Was-droogcombinaties', 'Luchtbevochtigers',
       'Luchtreinigers', 'Robotstofzuigers', 'Kruimeldieven',
       'IP-cameras', 'Alarmsystemen', 'Deursloten', 'Strijkijzers',
       'Strijkplanken', 'Kledingstomers', 'Strijkplankhoezen', 'Wekkers',
       'Babyfoons', 'Vaatwassers', 'Koelkasten', 'Diepvriezers',
       'Microgolfovens', 'Ovens', 'Fornuizen', 'Kookplaten', 'Pannensets',
       'Keukenrobots', 'Blenders', 'Foodprocessors', 'Staafmixers',


In [77]:
df_v5.to_csv('/Users/bert/Desktop/comparison_bol_coolblue/data/processed/coolblue.csv')